# Web Scraping with Beautiful Soup

* * *

### Icons used in this notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
⚠️ **Warning**: Heads-up about tricky stuff or common mistakes.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>

### Learning Objectives
1. [Reflection: To Scape Or Not To Scrape](#when)
2. [Extracting and Parsing HTML](#extract)
3. [Scraping the Illinois General Assembly](#scrape)

<a id='when'></a>

# To Scrape Or Not To Scrape

When we'd like to access data from the web, we first have to make sure if the website we are interested in offers a Web API. Platforms like Twitter, Reddit, and the New York Times offer APIs. **Check out D-Lab's [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) workshop if you want to learn how to use APIs.**

However, there are often cases when a Web API does not exist. In these cases, we may have to resort to web scraping, where we extract the underlying HTML from a web page, and directly obtain the information we want. There are several packages in Python we can use to accomplish these tasks. We'll focus two packages: Requests and Beautiful Soup.

Our case study will be scraping information on the [state senators of Illinois](http://www.ilga.gov/senate), as well as the [list of bills](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) each senator has sponsored. Before we get started, peruse these websites to take a look at their structure.

## Installation

We will use two main packages: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Go ahead and install these packages, if you haven't already:

In [ ]:
# Instala el paquete 'requests', que permite hacer solicitudes HTTP en Python.
# El prefijo %pip se usa en notebooks (como Google Colab) para ejecutar comandos de instalación de paquetes directamente desde la celda.
%pip install requests

In [ ]:
# Instala la biblioteca 'beautifulsoup4', que se utiliza para analizar (parsear) documentos HTML y XML.
# Es muy útil para tareas de scraping o extracción de datos de páginas web.
%pip install beautifulsoup4

We'll also install the `lxml` package, which helps support some of the parsing that Beautiful Soup performs:

In [ ]:
# Instala el paquete 'lxml', un parser rápido y eficiente para analizar documentos HTML y XML.
# BeautifulSoup puede usar 'lxml' como motor de análisis para obtener mejores resultados en velocidad y precisión.
%pip install lxml

In [ ]:
# Import required libraries
# Se importan las librerías necesarias para realizar scraping web:
# BeautifulSoup para analizar el contenido HTML, requests para hacer solicitudes HTTP,
# datetime para manejar fechas y horas, y time para controlar pausas en la ejecución.
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracting and Parsing HTML

In order to succesfully scrape and analyse HTML, we'll be going through the following 4 steps:
1. Make a GET request
2. Parse the page with Beautiful Soup
3. Search for HTML elements
4. Get attributes and text of these elements

## Step 1: Make a GET Request to Obtain a Page's HTML

We can use the Requests library to:

1. Make a GET request to the page, and
2. Read in the webpage's HTML code.

The process of making a request and obtaining a result resembles that of the Web API workflow. Now, however, we're making a request directly to the website, and we're going to have to parse the HTML ourselves. This is in contrast to being provided data organized into a more straightforward `JSON` or `XML` output.

In [ ]:
# Realiza una solicitud GET a la URL especificada para obtener el contenido de la página web,
# guarda el texto HTML de la respuesta en la variable 'src' y muestra los primeros 1000 caracteres para una vista previa.
# Esto permite verificar que la solicitud fue exitosa y se obtuvo contenido.

# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

<!DOCTYPE html>
<html lang="en">
<head id="Head1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="content-type" content="text/html;charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
    <meta charset="UTF-8">
    <!-- Meta Description -->
    <meta name="description" content="Welcome to the official government website of the Illinois General Assembly">
    <meta name="contactName" content="Legislative Information System">
    <meta name="contactOrganization" content="LIS Staff Services">
    <meta name="contactStreetAddress1" content="705 Stratton Office Building">
    <meta name="contactCity" content="Springfield">
    <meta name="contactZipcode" content="62706">
    <meta name="contactNetworkAddress" content="webmaster@ilga.gov">
    <meta name="contactPhoneNumber" content="217-782-3944">
    <meta name="contactFaxNumber" content="217-524-6059">
    <meta name


## Step 2: Parse the Page with Beautiful Soup

Now, we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

If you run into an error about a parser library, make sure you've installed the `lxml` package to provide Beautiful Soup with the necessary parsing tools.

In [ ]:
# Convierte el contenido HTML en un objeto BeautifulSoup usando el parser 'lxml',
# lo que facilita la navegación y extracción de datos del árbol HTML.
# Luego imprime una versión formateada y legible (prettify) de los primeros 1000 caracteres del HTML.

# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head id="Head1">
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta charset="utf-8"/>
  <!-- Meta Description -->
  <meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
  <meta content="Legislative Information System" name="contactName"/>
  <meta content="LIS Staff Services" name="contactOrganization"/>
  <meta content="705 Stratton Office Building" name="contactStreetAddress1"/>
  <meta content="Springfield" name="contactCity"/>
  <meta content="62706" name="contactZipcode"/>
  <meta content="webmaster@ilga.gov" name="contactNetworkAddress"/>
  <meta content="217-782-3944" name="contactPhoneNumber"/>
  <meta content="217-524-6059" name="contactFaxNumber"/>
  <meta content="State Of Illinois" name="originatorJur

The output looks pretty similar to the above, but now it's organized in a `soup` object which allows us to more easily traverse the page.

## Step 3: Search for HTML Elements

Beautiful Soup has a number of functions to find useful components on a page. Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors

Let's search first for **HTML tags**.

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

In [ ]:
# Busca y extrae todas las etiquetas <a> (enlaces) del documento HTML,
# y luego muestra las primeras 10 para ver ejemplos de los enlaces encontrados.

# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="af" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-za"></span> Afrikaans
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="sq" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-al"></span> Albanian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="ar" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-ae"></span> Arabic
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="hy" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-am"></span> Armenian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="az" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-az"></span> Azerbaijani
            

Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object.

These two lines of code are equivalent:

In [ ]:
# Extrae todas las etiquetas <a> del HTML usando dos métodos equivalentes:
# find_all("a") y pasando directamente "a" al objeto soup.
# Luego imprime el primer enlace encontrado con ambos métodos para mostrar que son iguales.

a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>
<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>


How many links did we obtain?

In [ ]:
# Imprime la cantidad total de etiquetas <a> (enlaces) encontradas en el documento HTML.
print(len(a_tags))

270


That's a lot! Many elements on a page will have the same HTML tag. For instance, if you search for everything with the `a` tag, you're likely to get more hits, many of which you might not want. Remember, the `a` tag defines a hyperlink, so you'll usually find many on any given page.

What if we wanted to search for HTML tags with certain attributes, such as particular CSS classes?

We can do this by adding an additional argument to the `find_all`. In the example below, we are finding all the `a` tags, and then filtering those with `class_="sidemenu"`.

In [ ]:
# Busca y extrae las etiquetas <a> que tienen la clase CSS 'sidemenu',
# y luego muestra los primeros 5 resultados para revisar los enlaces del menú lateral.

# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

[]

A more efficient way to search for elements on a website is via a **CSS selector**. For this we have to use a different method called `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use `"a.sidemenu"` as a CSS selector, which returns all `a` tags with class `sidemenu`.

In [ ]:
# Usa un selector CSS para obtener todas las etiquetas <a> con la clase 'sidemenu',
# y muestra los primeros 5 elementos seleccionados.

# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

[]

## 🥊 Challenge: Find All

Use BeautifulSoup to find all the `a` elements with class `mainmenu`.

In [ ]:
# Aquí debes agregar tu código para continuar con el procesamiento o análisis del contenido HTML,
# como extraer información específica, filtrar datos, o realizar alguna acción con los elementos seleccionados.

# YOUR CODE HERE


## Step 4: Get Attributes and Text of Elements

Once we identify elements, we want the access information in that element. Usually, this means two things:

1. Text
2. Attributes

Getting the text inside an element is easy. All we have to do is use the `text` member of a `tag` object:

In [ ]:
# Selecciona todos los enlaces (<a>) con la clase 'sidemenu'.
# Si se encuentran, imprime el primer enlace y su tipo de dato;
# si no, muestra un mensaje indicando que no se encontraron enlaces con esa clase.

# Get all sidemenu links as a list
side_menu_links = soup.select("a.sidemenu")

if side_menu_links:
    first_link = side_menu_links[0]
    print(first_link)
    print('Class: ', type(first_link))
else:
    print("No se encontraron enlaces con la clase 'sidemenu'")


No se encontraron enlaces con la clase 'sidemenu'


It's a Beautiful Soup tag! This means it has a `text` member:

In [ ]:
# Imprime el texto visible del primer enlace encontrado con la clase 'sidemenu'.
print(first_link.text)

Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

💡 **Tip**: You can access a tag’s attributes by treating the tag like a dictionary:

In [ ]:
# Imprime el texto visible del primer enlace encontrado con la clase 'sidemenu'.
print(first_link.text)

## 🥊 Challenge: Extract specific attributes

Extract all `href` attributes for each `mainmenu` URL.

In [ ]:
# Aquí debes agregar tu código para continuar con el procesamiento o análisis del contenido HTML,
# como extraer información específica, filtrar datos, o realizar alguna acción con los elementos seleccionados.

# YOUR CODE HERE

<a id='scrape'></a>

# Scraping the Illinois General Assembly

Believe it or not, those are really the fundamental tools you need to scrape a website. Once you spend more time familiarizing yourself with HTML and CSS, then it's simply a matter of understanding the structure of a particular website and intelligently applying the tools of Beautiful Soup and Python.

Let's apply these skills to scrape the [Illinois 98th General Assembly](http://www.ilga.gov/senate/default.asp?GA=98).

Specifically, our goal is to scrape information on each senator, including their name, district, and party.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

In [ ]:
# Realiza una solicitud GET a una página específica del sitio del Senado de Illinois con un parámetro en la URL (?GA=98).
# Luego obtiene el contenido HTML de la respuesta y lo convierte en un objeto BeautifulSoup usando el parser 'lxml',
# para poder analizar y extraer información estructurada del HTML.

# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Search for the Table Elements

Our goal is to obtain the elements in the table on the webpage. Remember: rows are identified by the `tr` tag. Let's use `find_all` to obtain these elements.

In [ ]:
# Busca y obtiene todas las filas de tabla (<tr>) en el documento HTML.
# Luego muestra cuántas filas se encontraron, lo cual es útil para saber cuántos registros o entradas contiene la tabla.

# Get all table row elements
rows = soup.find_all("tr")
len(rows)

⚠️ **Warning**: Keep in mind: `find_all` gets *all* the elements with the `tr` tag. We only want some of them. If we use the 'Inspect' function in Google Chrome and look carefully, then we can use some CSS selectors to get just the rows we're interested in. Specifically, we want the inner rows of the table:

In [ ]:
# Utiliza el selector CSS 'tr tr tr' para encontrar todas las filas de tabla que están anidadas dentro de otras dos filas <tr>.
# Esto puede servir para encontrar contenido profundamente anidado en tablas HTML complejas.
# Luego imprime las primeras 5 filas encontradas para revisar su estructura.

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

It looks like we want everything after the first two rows. Let's work with a single row to start, and build our loop from there.

In [ ]:
# Selecciona una fila específica de la tabla (en este caso, la tercera fila)
# y la imprime de forma estructurada y legible usando 'prettify()',
# para examinar su contenido y estructura HTML.
example_row = rows[2]
print(example_row.prettify())

Let's break this row down into its component cells/columns using the `select` method with CSS selectors. Looking closely at the HTML, there are a couple of ways we could do this.

* We could identify the cells by their tag `td`.
* We could use the the class name `.detail`.
* We could combine both and use the selector `td.detail`.

In [ ]:
# Recorre y muestra todas las celdas <td> dentro de la fila seleccionada, permitiendo observar su contenido individual.
for cell in example_row.select('td'):
    print(cell)
print()

# Recorre y muestra solo los elementos que tienen la clase 'detail', sin importar su etiqueta HTML.
# Esto permite filtrar por clase específica dentro de la fila.
for cell in example_row.select('.detail'):
    print(cell)
print()

# Recorre y muestra únicamente las celdas <td> que también tienen la clase 'detail',
# combinando filtro por etiqueta y clase para mayor precisión.
for cell in example_row.select('td.detail'):
    print(cell)
print()

We can confirm that these are all the same.

In [ ]:
# Verifica que los tres métodos de selección (todas las celdas <td>, todos los elementos con clase 'detail',
# y todas las <td> con clase 'detail') devuelvan exactamente los mismos elementos.
# Si alguno difiere, se lanzará un error. Esto confirma que, en este caso, todos los <td> tienen la clase 'detail'.
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Let's use the selector `td.detail` to be as specific as possible.

In [ ]:
# Extrae solo las celdas <td> que tienen la clase 'detail' dentro de la fila seleccionada.
# Esto permite enfocarse únicamente en la información relevante contenida en esas celdas.

# Select only those 'td' tags with class 'detail'
detail_cells = example_row.select('td.detail')
detail_cells

Most of the time, we're interested in the actual **text** of a website, not its tags. Recall that to get the text of an HTML element, we use the `text` member:

In [ ]:
# Extrae solo el texto contenido dentro de cada celda de la lista 'detail_cells',
# creando una lista con los valores textuales de las celdas, sin las etiquetas HTML.
# Luego imprime la lista 'row_data' con el texto limpio de las celdas seleccionadas.

# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

Looks good! Now we just use our basic Python knowledge to get the elements of this list that we want. Remember, we want the senator's name, their district, and their party.

In [ ]:
# Imprime valores específicos de la lista 'row_data' que corresponden a ciertos datos del senador:
# el nombre (posición 0), el distrito (posición 3) y el partido político (posición 4).
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

## Getting Rid of Junk Rows

We saw at the beginning that not all of the rows we got actually correspond to a senator. We'll need to do some cleaning before we can proceed forward. Take a look at some examples:

In [ ]:
# Imprime el contenido sin procesar (HTML) de algunas filas específicas:
# la primera (índice 0), la segunda (índice 1) y la última (índice -1),
# para observar cómo varía la estructura entre las diferentes filas de la tabla.
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

When we write our for loop, we only want it to apply to the relevant rows. So we'll need to filter out the irrelevant rows. The way to do this is to compare some of these to the rows we do want, see how they differ, and then formulate that in a conditional.

As you can imagine, there a lot of possible ways to do this, and it'll depend on the website. We'll show some here to give you an idea of how to do this.

In [ ]:
# Mide y muestra la cantidad de elementos (generalmente celdas <td> o <th>) dentro de ciertas filas para comparar su estructura.
# Las primeras dos filas (posiblemente encabezados o filas irregulares) tienen menos elementos (filas "malas").
# Las filas posteriores tienen más elementos, indicando que contienen datos completos (filas "buenas").

# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

Perhaps good rows have a length of 5. Let's check:

In [ ]:
# Filtra las filas que tienen exactamente 5 elementos (celdas), asumiendo que estas son las filas "buenas" con datos completos.
# Luego imprime la primera, la penúltima y la última fila filtrada para verificar el contenido.
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

We found a footer row in our list that we'd like to avoid. Let's try something else:

In [ ]:
# Selecciona todas las celdas <td> con la clase 'detail' dentro de la tercera fila de la tabla,
# para enfocarse en las celdas que contienen información relevante o detallada.
rows[2].select('td.detail')

In [ ]:
# Muestra las celdas con clase 'detail' de una fila "mala" (la última), que posiblemente no contiene datos útiles,
# y luego las celdas de una fila "buena" (la sexta), que sí tienen información relevante.
# Después, filtra todas las filas que contienen al menos una celda con clase 'detail',
# quedándose solo con las filas que contienen datos útiles.
# Finalmente, imprime la primera y última fila filtrada para revisión.

# Bad row
print(rows[-1].select('td.detail'), '\n')

# Good row
print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

Looks like we found something that worked!

## Loop it All Together

Now that we've seen how to get the data we want from one row, as well as filter out the rows we don't want, let's put it all together into a loop.

In [ ]:
# Inicializa una lista vacía para almacenar la información de los senadores.
# Filtra las filas válidas que contienen celdas con clase 'detail' para evitar filas irrelevantes o vacías.
# Recorre cada fila válida y extrae el texto de las celdas con clase 'detail'.
# Luego asigna el nombre, distrito (convertido a entero) y partido político a variables específicas.
# Finalmente, crea una tupla con esa información y la agrega a la lista 'members'.

# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

In [ ]:
# Muestra la cantidad total de senadores almacenados en la lista 'members'.
# Se espera que el resultado sea 61, que sería el número correcto de miembros.

# Should be 61
len(members)

Let's take a look at what we have in `members`.

In [ ]:
# Imprime las primeras 5 tuplas de la lista 'members' para revisar los datos de los primeros senadores extraídos.
print(members[:5])

## 🥊  Challenge: Get `href` elements pointing to members' bills

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format.

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`.

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips:

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [ ]:
# Realiza una solicitud GET a la página web específica y obtiene el contenido HTML.
# Usa BeautifulSoup para parsear el HTML.
# Selecciona las filas anidadas con el selector CSS 'tr tr tr'.
# Filtra filas que contienen celdas con clase 'detail' para eliminar filas irrelevantes.
# Luego, recorre cada fila válida para extraer texto de las celdas relevantes,
# asigna nombre, distrito y partido, y prepara una variable 'full_path' (vacía por ahora).
# Finalmente, guarda toda la información en una tupla y la añade a la lista 'members'.

# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [ ]:
# Descomenta esta línea para mostrar las primeras 5 tuplas en la lista 'members' y verificar los datos extraídos.
# Uncomment to test
# members[:5]

## 🥊  Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator.

In [ ]:
# YOUR CODE HERE
  # Define una función que recibe una URL, realiza el scraping de los miembros del senado
# y devuelve una lista con la información de cada senador (nombre, distrito, partido, full_path).
def get_members(url):
    # Aquí deberías completar el código para:
    # - hacer la solicitud GET a la URL
    # - parsear el contenido con BeautifulSoup
    # - extraer y filtrar las filas relevantes
    # - obtener los datos necesarios y guardarlos en una lista
    # Finalmente, devolver la lista con los miembros.
    return [___]

In [ ]:
# Prueba la función 'get_members' con la URL especificada,
# almacena la lista de senadores en 'senate_members' y muestra la cantidad total obtenida.

# Test your code
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

## 🥊 Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [ ]:
# Define una función que recibe una URL, descarga y analiza el contenido HTML,
# busca todas las filas de una tabla y extrae información relevante de cada fila
# sobre proyectos de ley (bill_id, descripción, cámara, última acción y fecha de última acción).
# Finalmente, devuelve una lista con tuplas que contienen estos datos.

def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src, "lxml")  # Es mejor especificar el parser
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # YOUR CODE HERE
        # Aquí debes extraer los datos específicos de cada fila:
        # por ejemplo, usando row.select('td') para obtener las celdas,
        # y luego acceder a su texto o atributos.
        bill_id = None  # Reemplaza con la extracción real
        description = None
        chamber = None
        last_action = None
        last_action_date = None

        bill = (bill_id, description, chamber, last_action, last_action_date)
        bills.append(bill)
    return bills


In [ ]:
# Descomenta estas líneas para probar la función 'get_bills' usando la URL almacenada en la posición 3
# del primer miembro de 'senate_members' y mostrar las primeras 5 entradas de proyectos de ley extraídas.

# Uncomment to test your code
# test_url = senate_members[0][3]
# get_bills(test_url)[0:5]

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
# Aquí debes escribir el código que falta para realizar la tarea que necesitas,
# como crear una estructura, procesar datos o cualquier otra operación que estés implementando.
# Por ejemplo, podrías crear un diccionario para almacenar los proyectos de ley con sus índices:
# bills_dict = {index: bill for index, bill in enumerate(bills_list)}
# Reemplaza esta línea con el código que desees agregar.

# YOUR CODE HERE


In [ ]:
# Descomenta esta línea para acceder y mostrar la información del proyecto de ley con índice 52 en el diccionario 'bills_dict'.
# bills_dict[52]